# Home Credit Modeling

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import iqr, randint, uniform
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import Imputer, StandardScaler, RobustScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from skopt import gp_minimize
from skopt.plots import plot_convergence, plot_objective
import pickle
import gc

%load_ext autotime

gc.enable()
np.random.seed(235)

path = "/Users/dsaxton/home_credit_default/"

#### Load the data

In [48]:
df = pd.read_csv(path + "preprocessed_train.csv")

time: 24.7 s


In [49]:
sk_id_curr = df.pop("SK_ID_CURR")
y = df.pop("TARGET")

time: 748 ms


#### Select features

In [50]:
clf = lgb.LGBMClassifier(n_estimators=1000, num_leaves=23, subsample=0.5)
clf.fit(df, y)

lgb_cols = df.columns[clf.feature_importances_ > 0]
len(lgb_cols)

453

time: 1min 34s


In [51]:
var_imp = pd.DataFrame({"Feature": df.columns, "Importance": clf.feature_importances_})[["Feature", "Importance"]].sort_values("Importance", ascending=False)

time: 4.32 ms


#### View important features

In [52]:
var_imp.head(50)

,Feature,Importance
220,MAX_DAYS_CREDIT_ENDDATE,136
107,MIN_PREV_AMT_ANNUITY_24M,136
308,RANGE_DAYS_ENTRY_PAYMENT,135
344,REGION_RATING_CLIENT_W_CITY_DIV_REGION_POPULAT...,135
127,AVG_PREV_AMT_CREDIT_DIV_ANNUITY,133
318,AMT_GOODS_PRICE_DIV_AMT_INCOME_TOTAL,130
320,AMT_CREDIT_DIV_AVG_PREV_REQ_AMOUNT,129
79,DAYS_LAST_PHONE_CHANGE,129
149,AVG_PREV_RATE_DOWNPAYMENT,129
180,MAX_BUREAU_UTILIZATION_12M,129


time: 8.96 ms


#### Average importance over non-zero importance features

In [53]:
int(np.nanmean(var_imp["Importance"].where(var_imp["Importance"] > 0)))

48

time: 4.03 ms


#### Importance of selected features

In [54]:
features = ["MIN_PREV_PROP_APPROVED_12M"]

var_imp[var_imp["Feature"].isin(features)]

,Feature,Importance
290,MAX_PAYMENT_SIZE_12M,149


time: 5.67 ms


#### Features with zero importance

In [55]:
var_imp.query("Importance == 0")

,Feature,Importance
374,NAME_INCOME_TYPE_Maternity_leave,0
414,OCCUPATION_TYPE_Realty_agents,0
485,ORGANIZATION_TYPE_XNA,0
408,OCCUPATION_TYPE_IT_staff,0
486,ORGANIZATION_TYPE_nan,0
375,NAME_INCOME_TYPE_Pensioner,0
80,FLAG_DOCUMENT_2,0
481,ORGANIZATION_TYPE_Transport:_type_2,0
18,FLAG_EMAIL,0
377,NAME_INCOME_TYPE_Student,0


time: 18.1 ms


#### LightGBM CV

In [ ]:
params = {"n_estimators": 20000, 
          "num_leaves": 113, 
          "learning_rate": 0.0025, 
          "subsample": 0.5, 
          "colsample_bytree": 0.5, 
          "reg_lambda": 1}

lgb_data = lgb.Dataset(data=df[lgb_cols], 
                       label=y)

cv_result = lgb.cv(params=params, 
                   train_set=lgb_data, 
                   nfold=5, 
                   metrics="auc", 
                   early_stopping_rounds=200, 
                   stratified=True, 
                   shuffle=True, 
                   verbose_eval=100, 
                   show_stdv=True, 
                   seed=123)

cv_result = pd.DataFrame(cv_result)

/Users/dsaxton/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	cv_agg's auc: 0.760197 + 0.00292305
[200]	cv_agg's auc: 0.761587 + 0.00289455
[300]	cv_agg's auc: 0.763434 + 0.00288264


In [ ]:
cv_result.tail()

#### Fit chosen model

In [13]:
lgb_model = lgb.LGBMClassifier(n_estimators=11000, 
                         num_leaves=113, 
                         learning_rate=0.0025, 
                         subsample=0.5, 
                         colsample_bytree=0.5, 
                         reg_lambda=0.8)

lgb_model.fit(df[lgb_cols], y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
        learning_rate=0.005, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=5045,
        n_jobs=-1, num_leaves=113, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.9, silent=True, subsample=0.5,
        subsample_for_bin=200000, subsample_freq=1)

time: 18min 40s


#### Score test data and generate submission

In [14]:
df = pd.read_csv(path + "preprocessed_test.csv")

time: 3.44 s


In [15]:
df["NAME_INCOME_TYPE_Maternity_leave"] = 0

time: 12.4 ms


In [16]:
submission = pd.DataFrame({"SK_ID_CURR": df["SK_ID_CURR"], 
                           "TARGET": lgb_model.predict_proba(df[lgb_cols])[:,1]})

submission.head()

,SK_ID_CURR,TARGET
0,100001,0.028409
1,100005,0.131907
2,100013,0.022533
3,100028,0.034576
4,100038,0.182258


time: 11.2 s


In [17]:
submission.to_csv(path + "submission.csv", index=False)

time: 163 ms
